# PREPARAÇÃO DO AMBIENTE





In [ ]:
!pip install -U \
  transformers \
  accelerate \
  bitsandbytes \
  langchain-core \
  langchain-community \
  langgraph \
  python-dotenv \
  torch

import os
import re
import torch
from typing import TypedDict, List

from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain_community.llms import HuggingFacePipeline
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END

from google.colab import drive
from huggingface_hub import login

drive.mount('/content/drive')


## CARREGANDO O MODELO LORA

In [ ]:
ENV_PATH = "/content/drive/MyDrive/token-hf/env"
load_dotenv(ENV_PATH)

HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

HF_USER_REPO = os.getenv("HF_USER_REPO")
HF_REPO = f"{HF_USER_REPO}/assistente-medico-lora"

tokenizer = AutoTokenizer.from_pretrained(HF_REPO)

model = AutoModelForCausalLM.from_pretrained(
    HF_REPO,
    device_map="auto",
    torch_dtype=torch.float16
)


## PIPELINE HUGGINGFACE + LANGCHAIN LLM

In [ ]:
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=False,
    temperature=0.3,
    repetition_penalty=1.1,
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

## PREPARANDO O ASSISTENTE

In [ ]:
SYSTEM_PROMPT = """
Você é um assistente médico-científico. Responda exclusivamente em português.

Responda EXCLUSIVAMENTE com base no contexto fornecido.
Não utilize conhecimento externo.

REGRAS OBRIGATÓRIAS:
- A resposta DEVE conter APENAS duas linhas.
- A primeira linha DEVE começar exatamente com:
  "Decisão: SIM", "Decisão: NÃO" ou "Decisão: TALVEZ"
- A segunda linha DEVE começar exatamente com:
  "Justificativa:"
- A decisão final sempre deve ser de um médico.

PROIBIÇÕES ABSOLUTAS:
- NÃO inclua rótulos internos, nomes técnicos ou palavras como:
  "assistant.", "analysis", "context".
- NÃO repita a pergunta.
- NÃO inclua texto fora das duas linhas.
- Não responda em inglês.
- Não receite tratamentos ou medicamentos.
"""

def build_prompt(question: str, context: str) -> str:
    return f"""
            {SYSTEM_PROMPT}

            Pergunta:
            {question}

            Contexto científico:
            {context}

            Resposta:
            """.strip()

def extract_final_answer(raw_text: str) -> str:
    if not raw_text:
        return ""

    # Remove tudo antes de "Resposta:"
    if "Resposta:" in raw_text:
        raw_text = raw_text.split("Resposta:", 1)[-1]

    raw_text = raw_text.strip()

    # Captura decisão e justificativa mesmo se estiverem na mesma linha
    match = re.search(
        r"(Decisão:\s*(SIM|NÃO|TALVEZ))\s*(Justificativa:\s*.+)",
        raw_text,
        re.IGNORECASE | re.DOTALL
    )

    if not match:
        return ""

    decision = match.group(1).strip()
    justification = match.group(3).strip()

    return f"{decision}\n{justification}"

class AgentState(TypedDict):
    question: str
    context: str
    answer: str

def assistant_node(state: AgentState):
    prompt = build_prompt(state["question"], state["context"])
    response = llm.invoke(prompt).strip()
    response = extract_final_answer(response)

    if not validate_output(response):
        response = "Decisão: TALVEZ\nJustificativa: O contexto fornecido é insuficiente."

    return {"answer": response}

## GRAFO

In [ ]:
builder = StateGraph(AgentState)
builder.add_node("assistant", assistant_node)
builder.set_entry_point("assistant")
builder.add_edge("assistant", END)

graph = builder.compile()

## VALIDA SAÍDA

In [ ]:
def validate_output(raw_text: str) -> bool:

    extracted = raw_text

    if not extracted:
        return False

    lines = extracted.splitlines()

    if len(lines) != 2:
        return False

    if not lines[0].startswith("Decisão:"):
        return False

    if not any(x in lines[0] for x in ["SIM", "NÃO", "TALVEZ"]):
        return False

    if not lines[1].startswith("Justificativa:"):
        return False

    return True


## TESTE DE EXECUÇÃO

In [ ]:
DEFAULT_CONTEXT = (
    "O contexto a seguir representa evidências médico-científicas consolidadas, "
    "baseadas em estudos clínicos, revisões sistemáticas e diretrizes reconhecidas. "
    "As informações refletem consensos amplamente aceitos na literatura científica "
    "e devem ser utilizadas exclusivamente para responder à pergunta apresentada."
)

result = graph.invoke({
     "question": "O uso diário de protetor solar previne o câncer de pele em pessoas com alto risco de desenvolver a doença?",
     "context": f"{DEFAULT_CONTEXT}"
})

print(result["answer"])


## TESTE DO CHAT ASSISTENTE

In [ ]:
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END

class ChatState(TypedDict):
    messages: List[BaseMessage]

#---------------------------------------------------------

def assistant_node(state: ChatState) -> ChatState:
    question = state["messages"][-1].content

    prompt = build_prompt(question, DEFAULT_CONTEXT)
    response = llm.invoke(prompt).strip()
    response = extract_final_answer(response)

    return {
        "messages": state["messages"] + [AIMessage(content=response)]
    }

#----------------------------------------------------------

def run_assistant_chat():
    print("=" * 60)
    print("🩺 Assistente Médico-Científico")
    print("Digite sua pergunta ou 'sair' para encerrar.")
    print("=" * 60)

    state = {"messages": []}

    while True:
        user_input = input("\n👤 Pergunta: ").strip()

        if user_input.lower() in {"sair", "exit", "quit"}:
            print("\n👋 Encerrando o assistente.")
            break

        state["messages"].append(HumanMessage(content=user_input))

        result = app.invoke(state)

        answer = result["messages"][-1].content.strip()

        answer = extract_final_answer(answer)

        # Garantia mínima de saída limpa
        lines = [l for l in answer.splitlines() if l.strip()]
        print("\n🤖 Resposta:")
        if len(lines) >= 2:
            print(lines[0])
            print(lines[1])
        else:
            print(answer)

        state = result


graph = StateGraph(ChatState)

graph.add_node("assistant", assistant_node)
graph.set_entry_point("assistant")
graph.add_edge("assistant", END)

app = graph.compile()


run_assistant_chat()
